In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import preprocessing
from scipy import stats
import collections
import operator
import argparse
import random
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn import metrics
from statistics import mean 
from sklearn.naive_bayes import GaussianNB
from sklearn.utils import resample
from sklearn import preprocessing
from sklearn import utils
from sklearn.datasets import load_digits
from sklearn import svm
import xgboost as xgb
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import random
from sklearn.decomposition import PCA, KernelPCA, NMF

C:\Users\mltla\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\mltla\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\mltla\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
data = pd.read_csv("unscriptedML.csv")
#("aPromptTextFeatures.csv")
print(data.shape)

#binary labels
d10 = []
s1 = []
for i in range(0, data.shape[0]):
    if int(data.phq9[i]) >= 10:
        d10.append(1)
    else:
        d10.append(0)
    if int(data.q9[i]) >= 1:
        s1.append(1)
    else:
        s1.append(0)
data["phq9"] = d10
data["q9"] = s1
data.head()

(110, 2271)


,id,phq9,q9,F0final_sma_f0v_duration,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,...,logHNR_sma_de_risetime,F0final_sma_f0v_nnz,F0final_sma_f0v_meanSegLen,F0final_sma_f0v_maxSegLen,F0final_sma_f0v_minSegLen,F0final_sma_f0v_segLenStddev,F0final_sma_f0p_meanSegLen,F0final_sma_f0p_maxSegLen,F0final_sma_f0p_minSegLen,F0final_sma_f0p_segLenStddev
0,8181,0,0,29.72,0.053588,0.121533,0.279239,0.067945,0.157706,0.225651,...,0.428375,0.399394,0.237500,1.20,0.02,0.192370,0.368723,3.18,0.02,0.611432
1,8170,0,0,19.12,0.014378,0.023862,0.076276,0.009483,0.052414,0.061898,...,0.448418,0.288180,0.167742,0.57,0.03,0.124323,0.428387,4.53,0.02,0.821184
2,4041,1,0,29.60,0.054657,0.080013,0.177709,0.025356,0.097696,0.123052,...,0.444540,0.318919,0.238947,0.79,0.03,0.188216,0.519474,3.48,0.02,0.737228
3,8650,1,0,29.82,0.140065,0.247997,0.375071,0.107933,0.127074,0.235007,...,0.432584,0.525486,0.210704,0.58,0.02,0.149160,0.189014,1.94,0.02,0.278452
4,4782,1,0,29.65,0.041752,0.075815,0.195711,0.034062,0.119896,0.153958,...,0.457752,0.326813,0.210455,0.77,0.03,0.154375,0.453023,1.90,0.02,0.491564


In [3]:
testids = [1607712777, 292, 2613, 1610640355, 1607494599, 1607040811, 1608492986, 6390, 396, 1607734901, 1607350992, 1608992344, 1609903202, 74, 7159, 4698, 7547, 4441, 1607097951, 8479, 8170, 4707, 7516, 1609174124, 1608853150, 8516, 1611424664, 2843, 1607040596, 1953, 1607772081, 1608564004, 2627, 1607217921, 1607118643, 1607314413, 1609887404, 1608335387, 4098, 1607046006, 1608242917, 8918, 1607131299, 9754, 1607262842, 1607273026, 2478, 1607536408, 1607291545, 1608707232, 1609941585, 1608200497, 1610630377, 7711, 1607810287, 9934, 1608850448, 4041, 1609166629, 1608168856, 1607572897, 6831, 1608586814, 1608588581, 2837, 8180, 1608631410, 1607051003, 3830, 4879, 1608920128, 1607019351, 8181, 3473, 1608335906, 1607738757, 1608770486, 7564, 1607495239, 1609983150, 1607397061, 1607696074, 103, 2222]
trainids = [4769, 1607928177, 1607269923, 7755, 4598, 1607807806, 1608741452, 3323, 1610110670, 1607133044, 9745, 1607291670, 5245, 4442, 319, 1607133218, 1607010270, 1608587203, 1609256130, 1608582258, 5028, 1609771771, 5229, 3517, 1608595561, 1608048050, 1607410780, 528, 1607134906, 3102, 1607555727, 1609887167, 3985, 7256, 3523, 1607289708, 1609890222, 850, 1608917024, 5047, 1608061691, 4782, 1608062276, 1056, 1611517276, 1607636681, 1607891972, 5571, 1609052616, 1607927243, 2525, 4353, 1610818662, 8640, 1607559849, 6706, 1608624428, 1607968838, 1608672132, 552, 1608537399, 1610381937, 1608607986, 381, 1608589576, 3920, 1608059746, 1609027319, 1607357022, 1607691623, 1609899907, 1608470962, 8791, 1610380419, 3064, 1609473849, 1607712704, 1609887249, 1609888813, 1608588103, 1244, 7279, 1607339125, 1607712682, 8472, 1269, 1607045076, 1607365865, 1846, 191, 1811, 1608702785, 1609049435, 5330, 1607257348, 1609890530, 3278, 1608586899, 1607939718, 2430, 1609893292, 60, 1607270186, 6336, 8650, 1608495626, 1608586953, 2121, 1607295286, 896, 1609889389, 1607560754, 6548, 6580, 1607440988, 1609111416, 1607807159, 8663, 1607129044, 6658, 1607799213, 3933, 1608596696, 1608663032, 1610791060, 1607135820, 1607413039, 1607659758, 1608487726, 4859, 1609142183, 1607276888, 7452, 1607368510, 1607266081, 2623, 1608416516, 2128, 3227, 5881, 6510, 1609166843, 7569, 1607712793, 1608850996, 3273, 1607939838, 9986, 3302, 1607206195, 1609082904, 1607510222, 7612, 1607022963, 1607051040, 1607719324, 1608849324, 1607642639, 1607104225, 705, 1608506424, 1608188073, 8018, 8085, 4755, 1611704179, 1607193886, 7007, 3041, 4001, 1552, 1716, 1608053349, 1608572299, 1608051417, 1607712784, 836, 1607929944, 1607795480, 1608200317, 415, 3662, 1610109929, 2496, 8550, 6868, 1608587385, 1608591490, 7370, 4549, 7505, 1879, 1876, 1608003341]

In [6]:
#train/test split
testdata = data[data['id'].isin(testids)]
print(testdata.shape)
traindata = data[data['id'].isin(trainids)]
print(traindata.shape)

(33, 2271)
(65, 2271)


In [7]:
#limit to features
testContent = testdata[testdata.columns[3:]]
print(testContent.shape)
trainContent = traindata[traindata.columns[3:]]
print(trainContent.shape)

(33, 2268)
(65, 2268)


In [8]:
#NEED TO SCALE BEFORE FEATURE SELECTION/REDUCATION

min_max_scaler = preprocessing.MinMaxScaler()  
np_scaled = min_max_scaler.fit_transform(trainContent)
featureSubset = pd.DataFrame(np_scaled)
np_scaled2 =  min_max_scaler.transform(testContent)
testSubset = pd.DataFrame(np_scaled2)
print(featureSubset.shape)
print(testSubset.shape)

(65, 2268)
(33, 2268)


In [9]:
label = "q9"

    
target = list(traindata[label])

KBest = SelectKBest(chi2, k=1).fit(featureSubset, target) 
f = KBest.get_support(1)
newDF = featureSubset[featureSubset.columns[f]].assign(target = target)

columns = trainContent.columns
names = list(columns[f])
print(f)
print(names)

[1374]
['F0final_sma_upleveltime50']


In [11]:
label = "q9"

    
target = list(traindata[label])

KBest = SelectKBest(chi2, k=1).fit(featureSubset, target) 
f = KBest.get_support(1)
newDF = featureSubset[featureSubset.columns[f]].assign(target = target)

columns = trainContent.columns
names = list(columns[f])
print(f)
print(names)

[589]
['pcm_fftMag_spectralHarmonicity_sma_quartile1']


In [10]:
print(featureSubset)
print(trainContent.columns)

        0         1         2         3         4         5         6     \
0   0.660050  0.169929  0.114493  0.103587  0.063318  0.054236  0.069838   
1   0.654023  0.049778  0.034673  0.051348  0.019983  0.051071  0.045037   
2   0.317618  0.093537  0.079683  0.078232  0.055936  0.048960  0.062326   
3   0.457994  0.252141  0.127735  0.121798  0.040612  0.069214  0.068387   
4   0.456576  0.114236  0.078402  0.085039  0.044379  0.060485  0.064295   
..       ...       ...       ...       ...       ...       ...       ...   
60  0.420418  0.201829  0.171465  0.497238  0.120101  0.596088  0.475448   
61  0.716767  0.177443  0.181578  0.198668  0.144604  0.134388  0.167856   
62  0.254165  0.244379  0.347591  0.593728  0.322556  0.574644  0.566174   
63  0.662531  0.093845  0.168607  0.210828  0.168318  0.165138  0.201564   
64  0.599078  0.247510  0.588447  0.695046  0.625848  0.498932  0.671840   

        7         8         9     ...      2258      2259      2260      2261  \
0   0.

In [21]:
traindata[["optimism", label]][:50]

,optimism,phq9
4,0.000000,0
5,0.000000,1
6,0.000000,1
7,0.000000,1
8,0.052632,0
10,0.006135,0
11,0.000000,1
12,0.000000,1
13,0.020408,1
14,0.000000,1


In [24]:
labels = ["phq9", "q9"]
featureEs = ["Chi", "PCA"]
modelTypelist = ["XG"]#["NB", "LR", "SVC1", "SVC2", "kNN3", "kNN5", "RF"]

for label in labels:
    
    target = list(traindata[label])
    
    for featureE in featureEs:
    
        featureDF = []
        testDFs = []

        if featureE == "Chi":
            nFeatureList = list(np.arange(1,11,1))

            for numberOfFeatures in nFeatureList:
                chisetup = SelectKBest(chi2, k=numberOfFeatures)
                chisetup = chisetup.fit(featureSubset, target)
                featureSubset2 = chisetup.transform(featureSubset)
                featureSubset2=pd.DataFrame(featureSubset2).assign(target = target)
                featureDF.append(featureSubset2)
                testSubset2 = chisetup.transform(testSubset)
                testDFs.append(pd.DataFrame(testSubset2))

        elif featureE == "PCA":
            nFeatureList = list(np.arange(1,11,1))
            for numberOfFeatures in nFeatureList:
                pca = PCA(n_components=numberOfFeatures)
                pca = pca.fit(featureSubset)
                X_pca = pca.transform(featureSubset)
                pcaDF = pd.DataFrame(X_pca)
                pcaDF = pcaDF.assign(target = target)
                featureDF.append(pcaDF)
                testSubset2 = pca.transform(testSubset)
                testDFs.append(pd.DataFrame(testSubset2))
                
                
        #create lists to populate
        flist = [] 
        mlist = []
        llist = []
        featureList = []
        f1List = []
        accuracyList = []
        truePosList = []
        trueNegList = []
        falsePosList = []
        falseNegList = []
        predictions = []
        rseed = []
        
        for f in range(0, len(featureDF)):
            print(featureE)
            print(f)
            
            train_phq9 = featureDF[f]
            X_test = testDFs[f]
            
            # upsampling 
            #Count 1s and 0s
            ones = len(train_phq9.loc[train_phq9['target'] == 1])
            zeros = len(train_phq9.loc[train_phq9['target'] == 0])
            if ones >= zeros:
                majority = 1
                minority = 0
            else:
                majority = 0
                minority = 1

            
            # Upsample TrainingSet 
            train_majority = train_phq9[train_phq9.target==majority]
            train_minority = train_phq9[train_phq9.target==minority]

            # Upsample minority class
            train_minority_upsampled = resample(train_minority, 
                                             replace=True,     # sample with replacement
                                             n_samples=len(train_majority),    # to match majority class
                                             random_state=42) # reproducible results

            # Combine majority class with upsampled minority class
            train_phq9 = pd.concat([train_majority, train_minority_upsampled])
            
            #seperate features and target
            y_train = train_phq9["target"]
            X_train = train_phq9.drop(columns = "target")
            
            
            print(X_train.head())
            print(X_test.head())
            
            for r in [1,2,4,8,16,32,64,128,256,512]:
                for modelType in modelTypelist:

                    #add data to lists
                    llist.append(label)
                    featureList.append(f +1)
                    flist.append(featureE)
                    mlist.append(modelType)

                    #chose model type
                    if modelType == "SVC1":
                        clf = svm.SVC(kernel='rbf', random_state=r)
                    elif modelType == "SVC2":
                        clf = svm.SVC(kernel='linear', random_state=r)
                    elif modelType == "RF":
                        clf = RandomForestClassifier(criterion="gini", max_depth=3, random_state=r)
                    elif modelType == "kNN3":
                        clf = KNeighborsClassifier(n_neighbors=3)
                    elif modelType == "kNN5":
                        clf = KNeighborsClassifier(n_neighbors=5)
                    elif modelType == "XG":
                        clf = xgb.XGBClassifier(max_depth=3, random_state=r)
                    elif modelType == "LR":
                        clf = LogisticRegression(random_state=r)
                    elif modelType == "NB":
                        clf = GaussianNB()


                    #train model and make predictions
                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    #evaluate model
                    conf_mat = confusion_matrix(list(testdata[label]), y_pred)
                    TN = conf_mat[0][0]
                    TP = conf_mat[1][1]
                    FP = conf_mat[0][1]
                    FN = conf_mat[1][0]
                    precision = TP/(TP+FP)
                    sensitivity = TP/(TP+FN)
                    f1 = (2*precision*sensitivity)/(precision + sensitivity)
                    accuracy = (TP+TN)/(TN+TP+FP+FN)

                    #populate lists with results
                    f1List.append(f1)
                    accuracyList.append(accuracy)
                    truePosList.append(TP)
                    trueNegList.append(TN)
                    falsePosList.append(FP)
                    falseNegList.append(FN)
                    predictions.append(y_pred)
                    rseed.append(r)

        resultsDF = pd.DataFrame()
        resultsDF["label"] = llist
        resultsDF["Engineering"] = flist
        resultsDF["model"] = mlist
        resultsDF["nFeatures"] = featureList
        resultsDF["F1"] = f1List
        resultsDF["Accuracy"] = accuracyList
        resultsDF["truePos"] = truePosList
        resultsDF["trueNeg"] = trueNegList
        resultsDF["falsePos"] = falsePosList
        resultsDF["falseNeg"] = falseNegList
        resultsDF["predictions"] = predictions
        resultsDF["randomSeed"] = rseed

        #resultsDF.to_csv("resultsAll/audio" + label + featureE + "XG.csv")

Chi
0
     0
3  0.0
4  0.0
5  0.0
6  0.0
7  0.0
     0
0  0.0
1  0.0
2  0.0
3  0.0
4  0.0


C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: Runtim

Chi
1
     0    1
3  0.0  0.0
4  0.0  0.0
5  0.0  0.0
6  0.0  0.0
7  0.0  0.0
     0    1
0  0.0  0.0
1  0.0  0.0
2  0.0  0.0
3  0.0  0.0
4  0.0  0.0


C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars


Chi
2
     0    1         2
3  0.0  0.0  0.000000
4  0.0  0.0  0.533333
5  0.0  0.0  0.500000
6  0.0  0.0  0.421053
7  0.0  0.0  0.695652
     0    1    2
0  0.0  0.0  0.0
1  0.0  0.0  0.0
2  0.0  0.0  0.0
3  0.0  0.0  0.0
4  0.0  0.0  0.0
Chi
3
     0    1         2    3
3  0.0  0.0  0.000000  0.0
4  0.0  0.0  0.533333  0.0
5  0.0  0.0  0.500000  0.0
6  0.0  0.0  0.421053  0.0
7  0.0  0.0  0.695652  0.0
     0    1    2    3
0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0
Chi
4
     0    1         2    3    4
3  0.0  0.0  0.000000  0.0  0.0
4  0.0  0.0  0.533333  0.0  0.0
5  0.0  0.0  0.500000  0.0  0.0
6  0.0  0.0  0.421053  0.0  0.0
7  0.0  0.0  0.695652  0.0  0.0
     0    1    2    3    4
0  0.0  0.0  0.0  0.0  0.0
1  0.0  0.0  0.0  0.0  0.0
2  0.0  0.0  0.0  0.0  0.0
3  0.0  0.0  0.0  0.0  0.0
4  0.0  0.0  0.0  0.0  0.0
Chi
5
     0       1    2         3    4    5
3  0.0  0.0000  0.0  0.000000  0.0  0.0
4  0.0  0.0000

...

C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: inva

PCA
8
          0         1         2         3         4         5         6  \
0  0.306254 -0.018760  0.296881 -0.326279 -0.101729 -0.355253 -0.012890   
1 -0.594326  0.819258  0.067845  0.146036  1.464522  0.841410  0.396087   
2  0.420846  0.453610  0.195812  0.542611  0.138056 -0.408898  0.308601   
3 -0.082890 -0.502929 -0.439120 -0.038872 -0.277138  0.509135  0.370858   
4 -0.227334 -0.306616 -0.091109 -0.351548 -0.194072 -0.850648  0.073217   

          7         8  
0 -0.478019  0.060323  
1 -0.302404 -0.243103  
2 -0.687666  0.804390  
3  0.109948 -0.117586  
4  0.017211 -0.572262  
          0         1         2         3         4         5         6  \
0  0.034842  0.493648 -0.026633 -0.101536 -0.348129 -0.197560 -0.083395   
1 -0.028592 -0.232153 -0.264256 -0.067037 -0.012374  0.208349  0.052556   
2 -0.371425  0.060738 -0.306814  0.010055  0.285413 -0.041855  0.307729   
3 -1.544675  2.922632  1.446323  0.623595  4.590311  3.263681  1.080061   
4 -0.460472  0.242307  0

C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:133: RuntimeWarning: inva

PCA
9
          0         1         2         3         4         5         6  \
0  0.306254 -0.018760  0.296881 -0.326279 -0.101729 -0.355253 -0.012890   
1 -0.594326  0.819258  0.067845  0.146036  1.464522  0.841410  0.396087   
2  0.420846  0.453610  0.195812  0.542611  0.138056 -0.408898  0.308601   
3 -0.082890 -0.502929 -0.439120 -0.038872 -0.277138  0.509135  0.370858   
4 -0.227334 -0.306616 -0.091109 -0.351548 -0.194072 -0.850648  0.073217   

          7         8         9  
0 -0.478019  0.060323  0.153355  
1 -0.302404 -0.243103  0.108856  
2 -0.687666  0.804390 -0.301933  
3  0.109948 -0.117586 -0.039328  
4  0.017211 -0.572262  0.355269  
          0         1         2         3         4         5         6  \
0  0.034842  0.493648 -0.026633 -0.101536 -0.348129 -0.197560 -0.083395   
1 -0.028592 -0.232153 -0.264256 -0.067037 -0.012374  0.208349  0.052556   
2 -0.371425  0.060738 -0.306814  0.010055  0.285413 -0.041855  0.307729   
3 -1.544675  2.922632  1.446323  0.6235

C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: RuntimeWarning: invalid value encountered in longlong_scalars
C:\Users\mltla\Anaconda3\lib\site-packages\ipykernel_launcher.py:131: Runtim